In [494]:
import pandas as pd

In [495]:
#Global Variables

# File names
## Import
csvFile = 'vgsales-12-4-2019.csv'

## Export
excelFileName = 'ficheiro.xlsx'
tab1Name = 'Raw Data'
tab2Name = 'Platform Rank'

# Pandas CSV configuration
encodingName = 'utf8'
separator = ','

# Pandas XLS configuration
engineName = 'xlsxwriter'


In [496]:
# Essa função é reponsável pelo load de um ficheiro CSV 
def extract_base(inCSVName):
    load_file = pd.read_csv(inCSVName,sep=separator,encoding=encodingName)
    return load_file
        

In [497]:
# Essa função é responsável por remover a primeira coluna de index do ficheiro
# o parâmetro axis=1 indica que o drop é na coluna.
def transform_remove_index(inFile):
    return inFile.drop(inFile.columns[0], axis=1)

In [498]:
# Essa função é responsável por criar uma porcentagem de linhas null
def count_null(inFile,inColumn):
    totalRows = len(inFile)
    countNull = 0
    for i, row in inFile.iterrows():
        if pd.isnull(row[inColumn]):
            countNull += 1
    porcent = round((countNull * 100) / totalRows,2)
    return porcent


In [499]:
# Essa função converte uma coluna para int, mas antes preenche null values como 0
def convert_int(inFile,inColumn):
    inFile[inColumn] = inFile[inColumn].fillna(0)
    inFile[inColumn] = inFile[inColumn].astype(int)
    return inFile

In [500]:
# Função responsável por fazer um group by na coluna Platform e somar os valores de Global_Sales
# O reset index cria indexes para a nova lista criada
# A função sort_values ordena os dados
def soma_global_sales_platform(inFile):
    file = inFile.groupby('Platform')['Global_Sales'].sum().reset_index()
    file = file.sort_values(by='Global_Sales', ascending = False).reset_index()
    return file

In [501]:
# Essa função recebe como parâmetro o pd.ExcelWriter, o file e a tab do excel e escreve no ficheiro
def write_xls(inWriter,inFile,inTab):
    inFile.to_excel(inWriter, sheet_name=inTab,index=False)
    

In [502]:
writer = pd.ExcelWriter(excelFileName, engine=engineName)

file = extract_base(csvFile)

percentNull = count_null(file,'ESRB_Rating')
file = convert_int(file,'Year')

file = transform_remove_index(file)
write_xls(writer,file,tab1Name)

maxGlobalPlatform = soma_global_sales_platform(file)
write_xls(writer,maxGlobalPlatform,tab2Name)

writer.close()


   Rank                           Name                      basename  \
0     1                     Wii Sports                    wii-sports   
1     2              Super Mario Bros.              super-mario-bros   
2     3                 Mario Kart Wii                mario-kart-wii   
3     4  PlayerUnknown's Battlegrounds  playerunknowns-battlegrounds   
4     5              Wii Sports Resort             wii-sports-resort   

      Genre ESRB_Rating Platform         Publisher         Developer  \
0    Sports           E      Wii          Nintendo      Nintendo EAD   
1  Platform         NaN      NES          Nintendo      Nintendo EAD   
2    Racing           E      Wii          Nintendo      Nintendo EAD   
3   Shooter         NaN       PC  PUBG Corporation  PUBG Corporation   
4    Sports           E      Wii          Nintendo      Nintendo EAD   

   VGChartz_Score  Critic_Score  ...  NA_Sales  PAL_Sales  JP_Sales  \
0             NaN           7.7  ...       NaN        NaN      